### 일자별 날씨정보 API
-아래사이트의 기상정보를 가져와서 dict 구조로 작성한후 json 파일로 저장하는 내용입니다.
-http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey=Y9vZZYdTITnGhBz7r3628UaVagdVfFEt7urjSsx3ha5rmD9ohXbuD546nfVlktbIbE9G5At1lXbcXKSo%2BEYYig%3D%3D&pageNo=1&numOfRows=10&dataType=XML&dataCd=ASOS&dateCd=DAY&startDt=20230101&endDt=20231231&stnIds=108


In [1]:
# !pip install pyOpenSSL
# !pip install request
# !pip install bs4
!pip install tqdm

In [4]:
###################
## 참고: 프로그레스바(진행바)
##################
sum=0
for x in tqdm(range(1000)):
    sum+=x

#밑에 셀 돌리고 이거 돌려야 에러안남 

100%|██████████| 1000/1000 [00:00<00:00, 1002223.18it/s]


In [3]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from tqdm import tqdm  #진행사항을 알리는 프로그레스바


In [5]:
########################################
### 전체데이터가 몇개인지 미리 알고 그 갯수를 전체데이터갯수에 넣으면 모든 자료 읽어올수 있음.
#######################################

url='http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey=Y9vZZYdTITnGhBz7r3628UaVagdVfFEt7urjSsx3ha5rmD9ohXbuD546nfVlktbIbE9G5At1lXbcXKSo%2BEYYig%3D%3D&pageNo=1&numOfRows=10&dataType=XML&dataCd=ASOS&dateCd=DAY&startDt=20230101&endDt=20231231&stnIds=108'
xml=requests.get(url)
soup=bs(xml.text,'html.parser')
totCnt=soup.totalcount.text
totCnt
#365 : totCnt에 저장된 '365'는 API 응답에서 추출한 전체 데이터 건수를 나타냅니다. 
# 이 값은 데이터포털 API에서 기상 관측 정보를 연도 단위로 제공하고 있는 것으로 보입니다. '365'는 연간(365일) 데이터가 있는 것으로 추정됩니다.

c:\Users\bluecom010\miniconda3\envs\OpenAI\Lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


'365'

In [6]:
# 공공API 인증키 (부성순강사 인증키임. 본인자료는 다시 인증키 받아야함.)
def myUrlOpen(지점코드=108,totCnt=1,startDate='20230101',endDate='20231231'):
    인증키='Y9vZZYdTITnGhBz7r3628UaVagdVfFEt7urjSsx3ha5rmD9ohXbuD546nfVlktbIbE9G5At1lXbcXKSo%2BEYYig%3D%3D'
    지점코드=108   #서울
    url='http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey='
    url=url+인증키
    url=url+ '&pageNo=1&numOfRows='
    url=url+str(totCnt)+'&dataType=XML&dataCd=ASOS&dateCd=DAY&startDt='
    url=url+startDate+'&endDt='+endDate+'&stnIds='
    url=url+str(지점코드)
    print(url)
    return url
# totCnt=5하면 <item> 5개보이고 / totCnt=365하면 <item> 365개 다 보임 


In [13]:
자료=[]
지점리스트={'지점명':['서울','부산','제주'],
              '지점코드': [108,253,184]} #기상청02_지상(종관,ASOS)일자료_조회서비스_오픈API활용가이드에 지점코드 나와있음
jijumList,dataList,minList,maxList,rnList=[],[],[],[],[]

for 지점명,지점코드 in tqdm(zip(지점리스트['지점명'],지점리스트['지점코드'])):

    url=myUrlOpen(지점코드,totCnt)
    xml=requests.get(url) #requests.get 함수를 사용하여 API에 GET 요청을 보내고, 응답을 받아 xml 변수에 저장
    soup=bs(xml.text,'html.parser') #받아온 응답(XML 형식)을 BeautifulSoup을 사용하여 파싱합니다
                                    #'html.parser'는 HTML 및 XML 문서를 파싱하는 데 사용되는 파서입니다. 
    data=soup.find_all('item') #파싱된 HTML 문서에서 'item' 태그를 가진 모든 요소를 찾아 리스트로 반환합니다.

#파싱 : 구조화되지 않은 데이터를 읽고 그것을 분석 가능한 형태로 변환하는 과정
#       예를 들어, 웹 페이지의 HTML 문서에서 특정 태그에 있는 데이터를 추출하거나, XML 형식의 데이터에서 원하는 요소를 선택하는 것이 파싱의 일부입니다. 
#       BeautifulSoup과 같은 라이브러리는 파싱을 쉽게 수행할 수 있도록 도와주는 도구 중 하나입니다.

    for item in data:
        jijum=지점명
        date=item.find('tm').text
        min=item.find('minta').text
        max=item.find('maxta').text
        rn=item.find('sumrn').text

        jijumList.append(jijum) ;dataList.append(date) ; maxList.append(max); minList.append(min) ; rnList.append(rn)

dataList=dict({'j':jijumList,
          'd':dataList,
          'm':maxList,
          'n':minList,
          'r':rnList})

# print(dataList)

0it [00:00, ?it/s]

http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey=Y9vZZYdTITnGhBz7r3628UaVagdVfFEt7urjSsx3ha5rmD9ohXbuD546nfVlktbIbE9G5At1lXbcXKSo%2BEYYig%3D%3D&pageNo=1&numOfRows=365&dataType=XML&dataCd=ASOS&dateCd=DAY&startDt=20230101&endDt=20231231&stnIds=108


c:\Users\bluecom010\miniconda3\envs\OpenAI\Lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
1it [00:03,  3.65s/it]

http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey=Y9vZZYdTITnGhBz7r3628UaVagdVfFEt7urjSsx3ha5rmD9ohXbuD546nfVlktbIbE9G5At1lXbcXKSo%2BEYYig%3D%3D&pageNo=1&numOfRows=365&dataType=XML&dataCd=ASOS&dateCd=DAY&startDt=20230101&endDt=20231231&stnIds=108


2it [00:04,  2.29s/it]

http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey=Y9vZZYdTITnGhBz7r3628UaVagdVfFEt7urjSsx3ha5rmD9ohXbuD546nfVlktbIbE9G5At1lXbcXKSo%2BEYYig%3D%3D&pageNo=1&numOfRows=365&dataType=XML&dataCd=ASOS&dateCd=DAY&startDt=20230101&endDt=20231231&stnIds=108


3it [00:06,  2.08s/it]


In [9]:
print(지점리스트['지점명'])
fileName='_'.join(지점리스트['지점명'])   # 파일명으로 사용하기 위하여 '서울_부산_제주'변수 저장
fileName

['서울', '부산', '제주']


'서울_부산_제주'

In [11]:
#########################3
## 판다스로제작후 저장
#########################
df=pd.DataFrame(dataList)
df.to_csv('c:/data/weather/날씨_'+fileName+'.csv', encoding='utf-8',index=False)
df.tail() #마지막 5행 출력 

,j,d,m,n,r
1090,제주,2023-12-27,6.8,-2.8,
1091,제주,2023-12-28,4.4,-1.7,
1092,제주,2023-12-29,4.6,-1.5,
1093,제주,2023-12-30,2.1,-0.9,13.1
1094,제주,2023-12-31,4.3,0.6,4.7


In [12]:
################
### json 파일로 저장
##################
import json
with open('c:/data/weather/날씨_'+fileName+'.json', 'w', encoding='utf-8') as f :
	json.dump(dataList, f, indent='\t',ensure_ascii=False)
